In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

all_teams = []

base_url = 'https://fbref.com'

# Get Premier League main page
html = requests.get('https://fbref.com/en/comps/9/Premier-League-Stats').text
soup = BeautifulSoup(html, 'lxml')

# Get first table containing teams
table = soup.find_all('table', class_='stats_table')[0]

# Parse rows to get team id and name together
rows = table.find_all('tr')
for row in rows:
    a_tag = row.find('a')
    if not a_tag:
        continue
    href = a_tag.get('href')
    if href and '/squads/' in href:
        parts = href.split('/')
        if len(parts) < 5:
            continue
        team_id = parts[3]
        team_name_raw = parts[-1]
        team_name = team_name_raw.replace('-Stats', '')
        
        # Construct team URL for 2024–2025 season
        team_url = f"{base_url}/en/squads/{team_id}/2024-2025/{team_name}-Stats"
        print(f"Scraping {team_url}")
        
        # Get team page
        data = requests.get(team_url).text
        soup_team = BeautifulSoup(data, 'lxml')

        # Get all stats tables on the page
        stats_tables = soup_team.find_all('table', class_='stats_table')
        if not stats_tables:
            print(f"No stats table found for {team_url}")
            continue

        stats_table = stats_tables[0]

        # Remove <tfoot> section (which contains "Squad Total", "Opponent Total")
        tfoot = stats_table.find('tfoot')
        if tfoot:
            tfoot.decompose()  # This removes it from the BeautifulSoup tree

        # Parse cleaned table using pandas
        filtered_table_html = str(stats_table)
        team_data = pd.read_html(filtered_table_html)[0]
        team_data['Team'] = team_name
        all_teams.append(team_data)

        time.sleep(5)  # Be nice to the server

# Combine all team data into one DataFrame
stat_df = pd.concat(all_teams, ignore_index=True)


Scraping https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats
Scraping https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats
Scraping https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats
Scraping https://fbref.com/en/squads/cd051869/2024-2025/Brentford-Stats
Scraping https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats
Scraping https://fbref.com/en/squads/943e8050/2024-2025/Burnley-Stats
Scraping https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats
Scraping https://fbref.com/en/squads/47c64c55/2024-2025/Crystal-Palace-Stats
Scraping https://fbref.com/en/squads/d3fd31cc/2024-2025/Everton-Stats
Scraping https://fbref.com/en/squads/fd962109/2024-2025/Fulham-Stats
Scraping https://fbref.com/en/squads/5bfb9659/2024-2025/Leeds-United-Stats
Scraping https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats
Scraping https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats
Scraping https://fbref.com/en/squads/19538

In [18]:
stat_df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Player             Nation                Pos   
0           David Raya             es ESP                 GK   
1       William Saliba             fr FRA                 DF   
2          Declan Rice            eng ENG                 MF   
3        Thomas Partey             gh GHA              MF,DF   
4     Leandro Trossard             be BEL                 FW   
..                 ...                ...                ...   
702    Tawanda Chirewa             zw ZIM                 MF   
703        Luke Cundle            eng ENG                 MF   
704         Tom Edozie            eng ENG                 MF   
705   Bastien Meupiyou             fr FRA                 DF   
706     Wesley Okoduwa            eng ENG                 FW   

    Unnamed: 3_level_0 Unnamed: 4_level_0 Playing Time                \
                   Age                 MP       Starts     Min   90s   
0                 28.0               38.0           38  3420.0  38.0   
1                 23.0               35.0           35  3039.0  33.8   
2                 25.0               35.0           33  2825.0  31.4   
3                 31.0               35.0           31  2797.0  31.1   
4                 29.0               38.0           28  2546.0  28.3   
..                 ...                ...          ...     ...   ...   
702               20.0                0.0            0     NaN   NaN   
703               22.0                0.0            0     NaN   NaN   
704               18.0                0.0            0     NaN   NaN   
705               18.0                0.0            0     NaN   NaN   
706               16.0                0.0            0     NaN   NaN   

    Performance       ... Per 90 Minutes                                  \
            Gls  Ast  ...           G-PK G+A-PK    xG   xAG xG+xAG  npxG   
0           0.0  0.0  ...           0.00   0.00  0.00  0.00   0.00  0.00   
1           2.0  0.0  ...           0.06   0.06  0.07  0.03   0.09  0.07   
2           4.0  7.0  ...           0.13   0.35  0.11  0.21   0.32  0.11   
3           4.0  2.0  ...           0.13   0.19  0.07  0.07   0.14  0.07   
4           8.0  7.0  ...           0.28   0.53  0.25  0.22   0.47  0.25   
..          ...  ...  ...            ...    ...   ...   ...    ...   ...   
702         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
703         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
704         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
705         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   
706         NaN  NaN  ...            NaN    NaN   NaN   NaN    NaN   NaN   

             Unnamed: 33_level_0                     Team Playing Time  
    npxG+xAG             Matches                                    MP  
0       0.00             Matches                  Arsenal          NaN  
1       0.09             Matches                  Arsenal          NaN  
2       0.32             Matches                  Arsenal          NaN  
3       0.14             Matches                  Arsenal          NaN  
4       0.47             Matches                  Arsenal          NaN  
..       ...                 ...                      ...          ...  
702      NaN             Matches  Wolverhampton-Wanderers          NaN  
703      NaN             Matches  Wolverhampton-Wanderers          NaN  
704      NaN             Matches  Wolverhampton-Wanderers          NaN  
705      NaN             Matches  Wolverhampton-Wanderers          NaN  
706      NaN             Matches  Wolverhampton-Wanderers          NaN  

[707 rows x 36 columns]

In [19]:
stat_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in stat_df.columns]

In [20]:
stat_df.rename(columns={
    'Unnamed: 0_level_0_Player': 'Player Name',
    'Unnamed: 1_level_0_Nation': 'Nation',
    'Unnamed: 2_level_0_Pos': 'Position',
    'Unnamed: 3_level_0_Age': 'Age',
    'Unnamed: 4_level_0_MP': 'Matches Played'
}, inplace=True)

In [21]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_,Playing Time_MP
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Arsenal,NaN
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Matches,Arsenal,NaN
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Matches,Arsenal,NaN
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Matches,Arsenal,NaN
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Matches,Arsenal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,Tawanda Chirewa,zw ZIM,MF,20.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers,NaN
703,Luke Cundle,eng ENG,MF,22.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers,NaN
704,Tom Edozie,eng ENG,MF,18.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers,NaN
705,Bastien Meupiyou,fr FRA,DF,18.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers,NaN


In [22]:
stat_df.loc[stat_df['Team_'].isin(['Brentford', 'Fulham']), 'Matches Played'] = stat_df.loc[stat_df['Team_'].isin(['Brentford', 'Fulham']), 'Playing Time_MP']

In [23]:
stat_df.drop(columns=["Playing Time_MP"], inplace=True)

In [24]:
stat_df

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches,Team_
0,David Raya,es ESP,GK,28.0,38.0,38,3420.0,38.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Arsenal
1,William Saliba,fr FRA,DF,23.0,35.0,35,3039.0,33.8,2.0,0.0,...,0.06,0.06,0.06,0.07,0.03,0.09,0.07,0.09,Matches,Arsenal
2,Declan Rice,eng ENG,MF,25.0,35.0,33,2825.0,31.4,4.0,7.0,...,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32,Matches,Arsenal
3,Thomas Partey,gh GHA,"MF,DF",31.0,35.0,31,2797.0,31.1,4.0,2.0,...,0.19,0.13,0.19,0.07,0.07,0.14,0.07,0.14,Matches,Arsenal
4,Leandro Trossard,be BEL,FW,29.0,38.0,28,2546.0,28.3,8.0,7.0,...,0.53,0.28,0.53,0.25,0.22,0.47,0.25,0.47,Matches,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,Tawanda Chirewa,zw ZIM,MF,20.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers
703,Luke Cundle,eng ENG,MF,22.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers
704,Tom Edozie,eng ENG,MF,18.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers
705,Bastien Meupiyou,fr FRA,DF,18.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Wolverhampton-Wanderers


In [ ]:
test_stat_df

In [8]:
stat_df.dropna(inplace=True)

In [9]:
stat_df.drop(columns=["Unnamed: 33_level_0_Matches"], inplace=True)

In [10]:
stat_df['League'] = 'English Premier League'

In [11]:
non_epl_teams = ['Sunderland', 'Burnley', 'Leeds-United']
stat_df.loc[stat_df['Team_'].isin(non_epl_teams), 'League'] = 'EFL Championship'

In [12]:
stat_df.rename(columns = {
    'Team_': 'Team'
}, inplace=True)

In [13]:
stat_df.rename(columns = {
    'League': 'League_Previous_Season'
}, inplace=True)

In [16]:
stat_df[stat_df['Team']== 'Fulham']

,Player Name,Nation,Position,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Team,League_Previous_Season


In [244]:
df = stat_df.copy()

In [248]:
df.drop(columns=["Playing Time_90s"], inplace=True)

In [250]:
df.rename(columns={
    'Playing Time_Start': 'Matches Started',
    'Playing Time_Min': 'Minutes Played',
    'Performance_Gls': 'Goals',
    'Performance_Ast': 'Assists',
    'Performance_G+A': 'Goals+Assists',
    'Performance_G-PK': 'Non-Penalty Goals',
    'Performance_PK': 'Penalties Scored',
    'Performance_PKatt': 'Penalties Taken',
    'Performance_CrdY': 'Yellow Cards',
    'Performance_CrdR': 'Red Cards',
    'Expected_xG': 'xG',
    'Expected_npxG': 'Non-Penalty xG (npxG)',
    'Expected_xAG': 'xAG',
    'Expected_npxG+xAG': 'npxG + xAG',
    'Progression_PrgC': 'Progressive Carries',
    'Progression_PrgP': 'Progressive Passes',
    'Progression_PrgR': 'Progrssive Passes Received'
}, inplace=True)

In [260]:
df.to_csv("final_epl_stats.csv")

In [29]:
import pandas as pd

# Load your original CSV
df = pd.read_csv("final_epl_stats.csv")

# Define the essential columns to keep
columns_to_keep = [
    "Player Name", "Nation", "Position", "Age",
    "Matches Played", "Playing Time_Starts", "Minutes Played", "Yellow Cards", "Red Cards",
    "Goals", "Assists", "Goals+Assists",
    "xG", "xAG", "npxG + xAG",
    "Per 90 Minutes_Gls", "Per 90 Minutes_Ast", "Per 90 Minutes_G+A",
    "Per 90 Minutes_xG", "Per 90 Minutes_xAG",
    "Team", "League_Previous_Season"
]

# Keep only those columns
df_reduced = df[columns_to_keep]

# Save the reduced CSV
df_reduced.to_csv("epl_stats.csv", index=False)